# **TESTE - Z PARA UMA AMOSTRA**
 Teste Z: geralmente amostras < 30
Esse teste é aplicado quando uma amostra aleatória de tamanho n for extraída de uma população com distribuição normal com média (µ)
desconhecida e desvio-padrão (o) conhecido.

Entao como entrada tenho:

       1) a media informada (nao eh a media populacional, mas uma media teorica ou de senso comum)

       2) o banco de dados em formato csv e a variavel a ser testada

       3) Preciso que os dados tenham distribuição Normal, por isso teste o Ks teste antes

  ***  No teste z irei comparar a media da minha amostra com a media informada***


In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches #para legendas manuais
import math
import scipy.stats as st
import random

TESTE DE NORMALIDADE

In [3]:
class KolmogorovSmirnovTest:
    def __init__(self, data_filename, target_column):
        #self.alpha = alpha
        self.data = pd.read_csv(data_filename)
        self.target_column = target_column
        self.prepare_data()
        self.calculate_expected()
        #self.run_test()


    def prepare_data(self):
        self.frequency_absolute = self.data[self.target_column].value_counts().reset_index()
        self.frequency_absolute.rename(columns={'index': 'X_i', self.target_column: 'Fabs'}, inplace=True)
        self.frequency_absolute = self.frequency_absolute.sort_values(by='X_i', ascending=True).reset_index(drop=True)
        self.frequency_absolute['Fac'] = np.cumsum(self.frequency_absolute['Fabs'])
        self.N_samples = sum(self.frequency_absolute['Fabs'])
        self.frequency_absolute['Fobs'] = self.frequency_absolute['Fac'] / self.N_samples
        mu = self.data[self.target_column].mean()
        sd = self.data[self.target_column].std()
        self.frequency_absolute['Zcalc'] = ((self.frequency_absolute['X_i'] - mu) / sd).round(2)

    def calculate_expected(self):
        m = len(self.frequency_absolute['Zcalc'])
        F_esp = []

        for z in range(0, m):
            value = st.norm.cdf(self.frequency_absolute['Zcalc'][z])
            F_esp.append(round(value, 4))

        self.frequency_absolute['Fesp'] = F_esp
        self.frequency_absolute['Exp(i)_Obs(i)'] = abs(np.array(F_esp) - self.frequency_absolute['Fobs'])

        Fesp_obs2 = [round(self.frequency_absolute['Fesp'][0], 4)]
        for i in range(1, len(F_esp)):
            value = F_esp[i] - self.frequency_absolute['Fobs'][i - 1]
            Fesp_obs2.append(value)

        self.frequency_absolute['Exp(i)_Obs(i-1)'] = Fesp_obs2

    def run_test(self):
        self.a_values = [0.05]

        max1 = max(self.frequency_absolute['Exp(i)_Obs(i)'])
        max2 = max(self.frequency_absolute['Exp(i)_Obs(i-1)'])
        self.D_calc = max(max1, max2)
        # Achar valor critico
        self.Dcrit_values = []  # Lista para armazenar os valores de Dcrit
        criticalV = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/kolmogorovCriticalvalues.csv")

        for alpha in self.a_values:
            if self.N_samples <= 35:
                row_index = criticalV[criticalV['n'] == self.N_samples].index[0]
                Dcrit = criticalV.loc[row_index][str(alpha)]
            else:
                match alpha:
                    case 0.2:
                        Dcrit = 1.07/math.sqrt(self.N_samples)
                    case 0.15:
                        Dcrit = 1.14/math.sqrt(self.N_samples)
                    case 0.1:
                        Dcrit = 1.22/math.sqrt(self.N_samples)
                    case 0.05:
                        Dcrit = 1.36/math.sqrt(self.N_samples)
                    case 0.01:
                        Dcrit = 1.63/math.sqrt(self.N_samples)
            self.Dcrit_values.append(round(Dcrit,3))  # Adicione o valor de Dcrit à lista

        self.resultado = []
        if(self.D_calc>Dcrit):
            print("Amostra nao segue distribuicao normal")
            return self.resultado == 0
        else:
            print("Amostra segue distribuicao normal")
            return self.resultado == 1



# **TESTE Z**

In [4]:
import random
import numpy as np
from scipy.stats import t
import pandas as pd
from scipy.stats import norm
import statistics
# Teste Z: geralmente amostras < 30
#Esse teste é aplicado quando uma amostra aleatória de tamanho n for extraída de uma população com distribuição normal com média (µ)
#desconhecida e desvio-padrão (o) conhecido
# O valor eh

class Teste_Z: # teste t media_info uma amostra
        def __init__(self, media_info, desvio, alpha, data_filename, variavel):
                self.alpha = alpha
                self.media_info = media_info
                # esse self.valor eh a media informadapopulacional que nao eh conhecida, que para o teste z nao eh conhecida
                self.desvio_populacional = desvio # desvio populacional dado
                self.data = pd.read_csv(data_filename)
                self.variavel = variavel # ----------------------- pensava que essa variavel era a mesma coisa de target column
                self.num_samples = len(self.data[self.variavel]) # --------------------------------------- não sei se seria assim

        def teste(self):

                self.media_amostra = self.data[self.variavel].mean()
                self.data = [valor for valor in self.data[self.variavel]] # transformar o df em lista para fazer os calculos

                # Calculando z_calculado
                erro_padrão = self.desvio_populacional/(self.num_samples)**(1/2)
                t_calculado = (self.media_info-self.media_amostra)/erro_padrão
                #aqui acima eu subtraio uma media informada, mas nao a real da media da amostra,
                #pos a media real eu nao conheco.
                #

                # Calculado Zc
                # normal seguindo (mi, s) = (0, 1)
                muz = self.media_info # media da amostra
                #desvio_padrao = 1 # desvio padrão
                pr = self.alpha/2 # Pois é bicaudal
                tc = t.ppf(pr, muz, erro_padrão)
                #aqui eh o erro padrao da amostra, q eh calculado a partir do desvio conhecido

                if t_calculado > tc:
                        print(f"Bicaudal: Como o valor {t_calculado} > {tc}")
                        print(f"aceitamos H0 a um nível de {self.alpha*100}% de significância")
                        print(f"Media amostral Nao difere da media informada")
                else:
                        print("Bicaudal: Rejeitamos H0 do valor", self.media_info) # Não tenho certeza se essa seria a frase
                        print(f"Media amostral difere da media informada")



# **VAMOS FAZER TRÊS TESTES ABAIXO**

**TESTE 1:**

In [5]:
df = '/content/gdrive/MyDrive/Colab Notebooks/maquinaAgric.csv'
ks = KolmogorovSmirnovTest(df, 'ProdMaq')
resultado = ks.run_test()
if (resultado == False): #NORMALIDADE ACEITA
    tt = Teste_Z(42.6, 5, 0.05, df, "ProdMaq") #vamos dizer q a media informada foi 42.6 e o desvio populacional e 5
    # o primeiro termo eh a media informada que irei comparar com a media da amostra
    # coloquei 42.6, mas posso colocar qualquer outro valor
    tt.teste()
else:
    print("Variavel nao segue uma distribuicao normal, considere um teste nao parametrico")


Amostra segue distribuicao normal
Bicaudal: Como o valor -0.04666666666666117 > -1.1839066135322995
aceitamos H0 a um nível de 5.0% de significância
Media amostral Nao difere da media informada


**TESTE 2:**

In [6]:
#PARA O MESMO DADO ACIMA, MUDANDO A MEDIA INFORMADA
tt = Teste_Z(5.6, 1, 0.05, df, "ProdMaq")
tt.teste()

Bicaudal: Rejeitamos H0 do valor 5.6
Media amostral difere da media informada


**TESTE 3:**
Banco de dados gerado aleatoriamente

In [7]:
df = '/content/gdrive/MyDrive/Colab Notebooks/randompd.csv'
ks = KolmogorovSmirnovTest(df, 'Variavel')
resultado = ks.run_test()
if (resultado == False): #NORMALIDADE ACEITA
    tt = Teste_Z(10.6, 2, 0.05, df, "Variavel")
    # o primeiro termo eh a media informada que irei comparar com a media da amostra
    # coloquei 42.6, mas posso colocar qualquer outro valor
    tt.teste()
else:
    print("Variavel nao segue uma distribuicao normal, considere um teste nao parametrico")



Amostra segue distribuicao normal
Bicaudal: Rejeitamos H0 do valor 10.6
Media amostral difere da media informada
